<a href="https://colab.research.google.com/github/Monalika-P/NLP-Implementations/blob/main/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier USsing Bidirectional LSTM

In [ ]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"monalikap","key":"8b61c9a0e70b9ce434e51b419e043a21"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c fake-news
!unzip -q /content/fake-news.zip
!rm /content/fake-news.zip

 95% 44.0M/46.5M [00:03<00:00, 20.6MB/s]
100% 46.5M/46.5M [00:03<00:00, 13.8MB/s]


In [ ]:
df=pd.read_csv('/content/train.csv',engine='python',error_bad_lines=False)

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [ ]:
df.shape

(20822, 5)

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
## get the independent features
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
## Check whether dataset is balanced or not
y.value_counts()

0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: label, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [ ]:
## vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print( messages['title'][i])
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # anything that is not a letter, replace it will blank space
    review = review.lower()
    review = review.split()
    print(review, type(review))

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    ## first iterate over the list 'review' and check if the word in the list review is not a stop word. If not a stop word, stem it
    review = ' '.join(review)
    corpus.append(review)
    print(corpus)
    print(i)
    #break

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

## Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 2068, 4743, 4581],
       [   0,    0,    0, ..., 3123, 3069, 2679],
       [   0,    0,    0, ..., 1900, 1035, 2591],
       ...,
       [   0,    0,    0, ..., 1800, 2901, 2499],
       [   0,    0,    0, ..., 4441, 2539, 1318],
       [   0,    0,    0, ..., 4016,  248, 2499]], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
131/131 [==============================] - 15s 22ms/step - loss: 0.3781 - accuracy: 0.8125 - val_loss: 0.2300 - val_accuracy: 0.8983
Epoch 2/10
131/131 [==============================] - 2s 13ms/step - loss: 0.1577 - accuracy: 0.9348 - val_loss: 0.2167 - val_accuracy: 0.9109
Epoch 3/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0754 - accuracy: 0.9751 - val_loss: 0.2756 - val_accuracy: 0.9100
Epoch 4/10
131/131 [==============================] - 2s 14ms/step - loss: 0.0402 - accuracy: 0.9883 - val_loss: 0.3096 - val_accuracy: 0.9061
Epoch 5/10
131/131 [==============================] - 2s 14ms/step - loss: 0.0172 - accuracy: 0.9959 - val_loss: 0.4017 - val_accuracy: 0.9085
Epoch 6/10
131/131 [==============================] - 2s 16ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.5232 - val_accuracy: 0.9002
Epoch 7/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 0.5790 - val_accuracy: 0.902

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1077,  104],
       [  99,  775]])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.9012165450121654
